In [ ]:
!pip install Korpora konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 729.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 49.3 MB/s eta 0:00:00


# 순환 신경망

## RNN

양방향 다층 신경망

- 입력 차원 : [배치 크기, 시퀀스 길이, 입력 특성 크기]
- 초기 은닉 상태 : [계층 수 * 양방향 여부 + 1, 배치 크기, 은닉 상태 크기]

In [ ]:
import torch
from torch import nn

input_size = 128
output_size = 256
num_layers = 3
bidirectional = True

model = nn.RNN(
    input_size=input_size,
    hidden_size=output_size,
    num_layers=num_layers,
    nonlinearity='tanh',
    bidirectional=bidirectional,
    batch_first=True
)

batch_size = 4
sequence_len = 6

inputs = torch.randn(batch_size, sequence_len, input_size)
h_0 = torch.rand(num_layers * (int(bidirectional) + 1), batch_size, output_size) # 초기 은닉 상태

outputs, hidden = model(inputs, h_0)

print(outputs.shape)
print(hidden.shape)

torch.Size([4, 6, 512])
torch.Size([6, 4, 256])


## LSTM

양방향 다층 LSTM

In [ ]:
input_size = 128
ouput_size = 256
num_layers = 3
bidirectional = True
proj_size = 64

model = nn.LSTM(
    input_size=input_size,
    hidden_size=ouput_size,
    num_layers=num_layers,
    batch_first=True,
    bidirectional=bidirectional,
    proj_size=proj_size,
)

batch_size = 4
sequence_len = 6

inputs = torch.randn(batch_size, sequence_len, input_size)
h_0 = torch.rand(
    num_layers * (int(bidirectional) + 1),
    batch_size,
    proj_size if proj_size > 0 else ouput_size,
)
c_0 = torch.rand(num_layers * (int(bidirectional) + 1), batch_size, ouput_size)

outputs, (h_n, c_n) = model(inputs, (h_0, c_0))

print(outputs.shape)
print(h_n.shape)
print(c_n.shape)

torch.Size([4, 6, 128])
torch.Size([6, 4, 64])
torch.Size([6, 4, 256])


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:879: UserWarning: LSTM with projections is not supported with oneDNN. Using default implementation. (Triggered internally at ../aten/src/ATen/native/RNN.cpp:1492.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


## 모델 실습

RNN과 LSTM을 활용해 문장 긍/부정 분류 모델을 학습

문장 분류 모델

In [ ]:
class SentenceClassifier(nn.Module):
  def __init__(self, n_vocab, hidden_dim, embedding_dim,
               n_layers, dropout=0.5, bidirectional=True, model_type='lstm'):
    super().__init__()

    self.embedding = nn.Embedding(
        num_embeddings=n_vocab,
        embedding_dim=embedding_dim,
        padding_idx=0
    )
    if model_type == "rnn":
      self.model = nn.RNN(
          input_size=embedding_dim,
          hidden_size=hidden_dim,
          num_layers=n_layers,
          bidirectional=bidirectional,
          dropout=dropout,
          batch_first=True,
      )
    elif model_type == "lstm":
      self.model = nn.LSTM(
          input_size=embedding_dim,
          hidden_size=hidden_dim,
          num_layers=n_layers,
          bidirectional=bidirectional,
          dropout=dropout,
          batch_first=True,
      )

    if bidirectional:
      self.classifier = nn.Linear(hidden_dim * 2, 1)
    else:
      self.classifier = nn.Linear(hidden_dim, 1)
    self.dropout = nn.Dropout(dropout)

  def forward(self, inputs):
    embeddings = self.embedding(inputs)
    output, _ = self.model(embeddings)
    last_output = output[:, -1, :]
    last_output = self.dropout(last_output)
    logits = self.classifier(last_output)
    return logits

데이터세트 불러오기

In [ ]:
import pandas as pd
from Korpora import Korpora


corpus = Korpora.load("nsmc")
corpus_df = pd.DataFrame(corpus.test)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 72.8MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 31.5MB/s]                           


In [ ]:
train = corpus_df.sample(frac=0.9, random_state=42)
test = corpus_df.drop(train.index)

print(train.head(5).to_markdown())
print("Training Data Size :", len(train))
print("Testing Data Size :", len(test))

|       | text                                                                                     |   label |
|------:|:-----------------------------------------------------------------------------------------|--------:|
| 33553 | 모든 편견을 날려 버리는 가슴 따뜻한 영화. 로버트 드 니로, 필립 세이모어 호프만 영원하라. |       1 |
|  9427 | 무한 리메이크의 소재. 감독의 역량은 항상 그 자리에...                                    |       0 |
|   199 | 신날 것 없는 애니.                                                                       |       0 |
| 12447 | 잔잔 격동                                                                                |       1 |
| 39489 | 오랜만에 찾은 주말의 명화의 보석                                                         |       1 |
Training Data Size : 45000
Testing Data Size : 5000


데이터 토큰화 및 단어 사전 구축

In [ ]:
from konlpy.tag import Okt
from collections import Counter


def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter()
    for tokens in corpus:
        counter.update(tokens)
    vocab = special_tokens
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    return vocab


tokenizer = Okt()
train_tokens = [tokenizer.morphs(review) for review in train.text]
test_tokens = [tokenizer.morphs(review) for review in test.text]

vocab = build_vocab(corpus=train_tokens, n_vocab=5000, special_tokens=["<pad>", "<unk>"])
token_to_id = {token: idx for idx, token in enumerate(vocab)}
id_to_token = {idx: token for idx, token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))

['<pad>', '<unk>', '.', '이', '영화', '의', '..', '가', '에', '...']
5002


정수 인코딩 및 패딩

In [ ]:
import numpy as np

def pad_sequences(sequences, max_length, pad_value):
  result = list()
  for sequence in sequences:
    sequence = sequence[:max_length]
    pad_length = max_length - len(sequence)
    padded_sequence = sequence + [pad_value] * pad_length
    result.append(padded_sequence)
  return np.asarray(result)

unk_id = token_to_id["<unk>"]
train_ids = [
    [token_to_id.get(token, unk_id) for token in review] for review in train_tokens
]
test_ids = [
    [token_to_id.get(token, unk_id) for token in review] for review in test_tokens
]

max_length = 32
pad_id = token_to_id["<pad>"]
train_ids = pad_sequences(train_ids, max_length, pad_id)
test_ids = pad_sequences(test_ids, max_length, pad_id)

print(train_ids[0])
print(test_ids[0])

[ 223 1716   10 4036 2095  193  755    4    2 2330 1031  220   26   13
 4839    1    1    1    2    0    0    0    0    0    0    0    0    0
    0    0    0    0]
[3307    5 1997  456    8    1 1013 3906    5    1    1   13  223   51
    3    1 4684    6    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


데이터로더 적용

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader


train_ids = torch.tensor(train_ids)
test_ids = torch.tensor(test_ids)

train_labels = torch.tensor(train.label.values, dtype=torch.float32)
test_labels = torch.tensor(test.label.values, dtype=torch.float32)

train_dataset = TensorDataset(train_ids, train_labels)
test_dataset = TensorDataset(test_ids, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

손실 함수와 최적화 함수 정의

In [ ]:
from torch import optim


n_vocab = len(token_to_id)
hidden_dim = 64
embedding_dim = 128
n_layers = 2

device = "cuda" if torch.cuda.is_available() else "cpu"
classifier = SentenceClassifier(
    n_vocab=n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim, n_layers=n_layers
).to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)

모델 학습 및 테스트

In [ ]:
def train(model, datasets, criterion, optimizer, device, interval):
    model.train()
    losses = list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval == 0:
            print(f"Train Loss {step} : {np.mean(losses)}")


def test(model, datasets, criterion, device):
    model.eval()
    losses = list()
    corrects = list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())
        yhat = torch.sigmoid(logits)>.5
        corrects.extend(
            torch.eq(yhat, labels).cpu().tolist()
        )

    print(f"Val Loss : {np.mean(losses)}, Val Accuracy : {np.mean(corrects)}")


epochs = 5
interval = 500

for epoch in range(epochs):
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device)

Train Loss 0 : 0.6867725849151611
Train Loss 500 : 0.6936247455859612
Train Loss 1000 : 0.6907391950086161
Train Loss 1500 : 0.682171862058684
Train Loss 2000 : 0.6720567100021853
Train Loss 2500 : 0.6646402294828337
Val Loss : 0.6168034291876772, Val Accuracy : 0.6782
Train Loss 0 : 0.5940654873847961
Train Loss 500 : 0.5844168995549817
Train Loss 1000 : 0.5647882052830288
Train Loss 1500 : 0.5538767893103105
Train Loss 2000 : 0.5439690747882413
Train Loss 2500 : 0.5374484636970922
Val Loss : 0.5012446052540606, Val Accuracy : 0.7622
Train Loss 0 : 0.554517388343811
Train Loss 500 : 0.5009422014632863
Train Loss 1000 : 0.4862860381513923
Train Loss 1500 : 0.4737898110847009
Train Loss 2000 : 0.46813520336377507
Train Loss 2500 : 0.4621158795046215
Val Loss : 0.44711719243861614, Val Accuracy : 0.8004
Train Loss 0 : 0.46445703506469727
Train Loss 500 : 0.410575522753055
Train Loss 1000 : 0.40422590316592394
Train Loss 1500 : 0.39952670239274457
Train Loss 2000 : 0.3960885977794205
Trai

학습된 모델로부터 임베딩 추출

In [ ]:
token_to_embedding = dict()
embedding_matrix = classifier.embedding.weight.detach().cpu().numpy()

for word, emb in zip(vocab, embedding_matrix):
  token_to_embedding[word] = emb

token = vocab[1000]
print(token, token_to_embedding[token])

보고싶다 [ 0.9156976  -0.55401576  1.8168696   1.2887877   0.02241418 -0.61677414
  0.09421384  1.2348244  -1.3410246   0.09005291  0.83459055  1.5599653
  0.09008738 -0.7722967  -1.0689512   0.40242195  0.5993975  -1.0171436
  0.76490223  1.1648107  -0.10444339  0.18582134  0.48415342  0.47307476
 -0.07117919  0.50694156 -0.02449991  0.8860589  -2.1261668   0.37267584
  1.395763   -0.22399338 -0.98970884  2.0817513   0.49492034  0.5672728
  0.80570793 -0.51452816 -0.2423634  -0.5295623  -2.767572   -0.60893476
 -0.50125694  1.2644466  -1.325586   -0.35699975  0.77019966  0.27758837
  1.6750653   0.88076276 -2.6860402   3.0295355   0.21739434  1.2575622
 -0.32314125 -0.18484998  0.9903218  -0.9300321  -0.75998497 -1.3938714
 -1.0250475  -0.4878564   1.892235    2.6940422  -0.02856987  0.06278264
  0.77608645 -0.6453776   0.01995367 -0.60925746 -0.9569977  -0.6779056
  1.0350417   0.9626615  -0.7517421   0.19573943 -1.3109112  -0.42941976
  0.5551294   1.6199611  -0.49342176  0.94042236  0.

### 사전 학습된 모델로 임베딩 계층 초기화

In [ ]:
from gensim.models import Word2Vec

tokens = [tokenizer.morphs(review) for review in corpus_df.text]

word2vec = Word2Vec(
    sentences=tokens,
    vector_size=128,
    window=5,
    min_count=1,
    sg=1,
    epochs=3,
    max_final_vocab=10000
)

In [ ]:
init_embedding = np.zeros((n_vocab, embedding_dim))

for index, token in id_to_token.items():
  if token not in ['<pad>', '<unk>']:
    init_embedding[index] = word2vec.wv[token]

embedding_layer = nn.Embedding.from_pretrained(
    torch.tensor(init_embedding, dtype=torch.float32)
)

In [ ]:
class SentenceClassifier(nn.Module):
    def __init__(
        self,
        n_vocab,
        hidden_dim,
        embedding_dim,
        n_layers,
        dropout=0.5,
        bidirectional=True,
        model_type="lstm",
        pretrained_embedding=None
    ):
        super().__init__()
        if pretrained_embedding is not None:
            self.embedding = nn.Embedding.from_pretrained(
                torch.tensor(pretrained_embedding, dtype=torch.float32)
            )
        else:
            self.embedding = nn.Embedding(
                num_embeddings=n_vocab,
                embedding_dim=embedding_dim,
                padding_idx=0
            )

        if model_type == "rnn":
            self.model = nn.RNN(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        elif model_type == "lstm":
            self.model = nn.LSTM(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )

        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2, 1)
        else:
            self.classifier = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings)
        last_output = output[:, -1, :]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

In [ ]:
classifier = SentenceClassifier(
    n_vocab=n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim,
n_layers=n_layers, pretrained_embedding=init_embedding
).to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)

epochs = 5
interval = 500

for epoch in range(epochs):
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device)

Train Loss 0 : 0.691292405128479
Train Loss 500 : 0.6692448214380565
Train Loss 1000 : 0.6627480921509502
Train Loss 1500 : 0.6475114267639602
Train Loss 2000 : 0.6250570234076969
Train Loss 2500 : 0.6136515822435846
Val Loss : 0.4809579539318054, Val Accuracy : 0.7774
Train Loss 0 : 0.47346651554107666
Train Loss 500 : 0.5059766391675153
Train Loss 1000 : 0.5005744232253714
Train Loss 1500 : 0.502079211249898
Train Loss 2000 : 0.49685681094115286
Train Loss 2500 : 0.4924142829052022
Val Loss : 0.46941806975835426, Val Accuracy : 0.7874
Train Loss 0 : 0.40222546458244324
Train Loss 500 : 0.4801936238230821
Train Loss 1000 : 0.4686541587307856
Train Loss 1500 : 0.46187163957411415
Train Loss 2000 : 0.4590513633190841
Train Loss 2500 : 0.4583448811126251
Val Loss : 0.426602516120996, Val Accuracy : 0.8016
Train Loss 0 : 0.666080892086029
Train Loss 500 : 0.43780473252375446
Train Loss 1000 : 0.4394672241631326
Train Loss 1500 : 0.4363539547006104
Train Loss 2000 : 0.4384947693806657
Trai